**Table of contents**<a id='toc0_'></a>    
- [pybulletとは](#toc1_)    
- [必要なライブラリのインポート](#toc2_)    
- [pybulletの起動](#toc3_)    
- [シミュレーションの初期設定](#toc4_)    
- [床の読み込み](#toc5_)    
- [物体の読み込み](#toc6_)    
- [ロボットの読み込み](#toc7_)    
- [カメラ位置の設定](#toc8_)    
- [デバック用の文字表示](#toc9_)    
- [シミュレーションの実行](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[pybulletとは](#toc0_)
pybulletは、物理シミュレーションエンジンの一つであり、ロボットの制御や物理シミュレーションを行うためのライブラリです。

ロボットのシミュレーションや制御にはROS（Robot Operating System）が使われることが多いですが、ROSは環境構築が複雑です。一方で、pybulletはpythonのライブラリなので、pythonが使える環境であれば簡単に扱うことができます。

こちらでは、基本的な機能のみを紹介します。  
pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。

# <a id='toc2_'></a>[必要なライブラリのインポート](#toc0_)

pybulletを使用する際は、`pybullet`をインポートする必要があります。   
また、pybulletに使用するファイルを読み込むために`import pybullet_data`もインポートします。

In [1]:
import pybullet
import pybullet_data

pybullet build time: Nov 28 2023 23:45:17


# <a id='toc3_'></a>[pybulletの起動](#toc0_)
pybulletを使用する際は、`pybullet.connect`を使用して、物理シミュレーションを行うためのサーバーを起動します。

サーバの種類には以下の種類が存在します
- `pybullet.GUI`
- `pybullet.DIRECT`
- `pybullet.SHARED_MEMORY`
- `pybullet.UDP`
- `pybullet.TCP`

基本的には、CUIで使用する場合は`pybullet.DIRECT`、GUIで使用する場合は`pybullet.GUI`を使用します。

<br>

今回はGUIを使用するため、`pybullet.GUI`を指定してサーバーを起動します。
以下セルを実行すると、pybulletのGUIが起動されます。  
（google colabなどではGUIが表示されないため、`pybullet.DIRECT`を指定してサーバーを起動する必要があります。）

In [2]:
physics_client = pybullet.connect(pybullet.GUI) 

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc4_'></a>[シミュレーションの初期設定](#toc0_)
シミュレーションを開始するにあたって、初期設定を行います。
- シミュレーション空間のリセット
- pybulletに必要なデータへのパスの追加
- 重力の設定
- 1stepあたりに経過する時間（秒単位）

In [3]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 1./240.
pybullet.setTimeStep(time_step) # 1stepあたりに経過する時間の設定

ven = Mesa
ven = Mesa


# <a id='toc5_'></a>[床の読み込み](#toc0_)
pybulletでは、標準で幾つかのモデルが用意されています。ここでは、その中で床が定義されているファイルである `plane.urdf`を読み込みます。  

---

urdfファイルは「ロボットモデルについて定義されるxml形式のファイル」になり、ロボットのリンクやジョイントについて定義できます。  
基本的には、ロボットに関して定義するものになりますが、`plane.urdf`では、ロボットではなく床のモデルが定義されています。

---

In [4]:
#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

# <a id='toc6_'></a>[物体の読み込み](#toc0_)
`createCollisionShape` `createVisualShape` `createMultiBody`関数を使用することで、任意のサイズのオブジェクト（ボックスや球など）

In [5]:
# ボックスの読み込み
## ボックスの重さ、サイズ、位置·姿勢を決める
mass = 5 # kg
box_size = [0.3, 0.3, 0.3]
position = [2, 0, 0.3]
orientation = [1, 0, 0, 0] # 四元数
box_collision_id = pybullet.createCollisionShape(pybullet.GEOM_BOX, halfExtents=box_size, physicsClientId=physics_client)
box_visual_id = pybullet.createVisualShape(pybullet.GEOM_BOX, halfExtents=box_size, physicsClientId=physics_client, rgbaColor=[1,0,0,1]) # 赤・半透明
box_body_id = pybullet.createMultiBody(mass, box_collision_id, box_visual_id, position, orientation, physicsClientId=physics_client)

# <a id='toc7_'></a>[ロボットの読み込み](#toc0_)
自身で定義したロボットのurdfファイルを読み込みます。
`loadURDF`関数の引数に「urdfファイルまでのパス」を指定することでロボットを生成することができます

In [6]:
# ロボットの読み込み
car_start_pos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
car_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
# urdfファイルのmeshはテクスチャが反映されないっぽいので各linkにrgbaタグで色を付けている
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf",car_start_pos, car_start_orientation)

# <a id='toc8_'></a>[カメラ位置の設定](#toc0_)
`resetDebugVisualizerCamera`関数で、GUIモードの際のカメラを設定できます。

In [7]:
# GUIモードの際のカメラの位置などを設定
camera_distance = 4.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

# <a id='toc9_'></a>[デバック用の文字表示](#toc0_)
`addUserDebugText`関数を使用することで、シミュレーション空間上の任意の位置にテキストを表示させることができます。  

他にも、`addUserDebugLine`では、シミュレーション空間上の任意の2点間を結ぶを結ぶ線を描画することができ、移動ロボットの移動経路やロボットアームの手先の軌跡などを描画する際に便利です。

このように、pybulletではシミュレーション結果を可視化するのに便利な機能が搭載されています。

In [8]:
# 画面上に文字を表示
text_position = [0.0, 0.0, 2.0]
life_time = 10.0 # 表示期間（秒）
pybullet.addUserDebugText(f"test text", text_position, textSize=2, lifeTime=life_time)

0

# <a id='toc10_'></a>[シミュレーションの実行](#toc0_)
`stepSimulation`関数を使用すると、`setTimeStep`で設定した時間分シミュレーション空間上で時間が経過します。  
ここでは、移動ロボットの両輪に速度指令を与えて200時刻分シミュレーションを実行しています。


In [9]:
import time
RIGHT_WHEEL_JOINT_IDX = 0
LEFT_WHEEL_JOINT_IDX = 1
for i in range(200):
    pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=10)
    pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=10)
    pybullet.stepSimulation()
    time.sleep(time_step)

: 